In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
import openai
import json
import csv
import pandas as pd

f = open("chatgptkey.txt", 'r')
line = f.readline()
openai.api_key = line




In [3]:
from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key
)

In [4]:
df1 = pd.read_csv('drinkdata.csv')
frame=pd.DataFrame(df1)


state를 실시간으로 저장하면서 진행하고 싶으니까 index와 배열 상태를 저장하면서 진행함


In [ ]:
import time
outputs=[]
failed=[]
rownum=0

In [67]:
print(len(outputs))
print(rownum)

148
148


In [66]:



for row in df1[rownum:]['텍스트 내용']:
  description =  row # 원하는 설명문을 입력하세요
  response = client.chat.completions.create(
  temperature=0.2,
  model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "주류 데이터를 검색하기 위한 태그를 설명문으로부터 추출하고 싶어. 술에 대한 정보를 포함하는 10자 이내의 태그와 각 태그의 중요도를 ,로 묶어서 출력하고 싶어. "},
    {"role": "system", "content": "또한 추출한 태그의 중요도를 1에서 10 사이의 수치로 표현해줘. 각 태그가 주관적인지 객관적인지 판단해서, 맛이나 향 등을 포함하거나 주관적이라면 높은 점수를 줘."},
    {"role": "user", "content": description},
  ]
  )

  if(rownum%5==0):
    time.sleep(30)
  print(response.choices[0].message.content)
  tempstr=response.choices[0].message.content
  if '태그: ' in tempstr and '중요도: ' in tempstr: #fine string
    # add tags
    lines=tempstr.split('\n')
    if "" in lines:
      lines.remove("")
    if len(lines)!=1 and '태그: ' in lines[0] and '중요도: ' in lines[1]:
      #good
      print("good")
      outputs.append(lines[0].replace("태그:","").replace(" ","")+"\n"+lines[1].replace("중요도:","").replace(" ",""))
    else:
      failed.append(rownum)
      outputs.append(tempstr.replace("태그:","").replace("중요도:","").replace(" ",""))
  else:
    failed.append(rownum)
    outputs.append(tempstr.replace("태그:","").replace("중요도:","").replace(" ",""))
  rownum=rownum+1



태그: 대윤가야곡주조, 가야곡왕주, 약주, 궁중술, 왕주, 역사, 전통, 청정지역, 깨끗한 맛, 숙취, 고급, 음식, 조청향, 단맛, 신맛, 목넘김, 쭈꾸미 볶음

중요도: 대윤가야곡주조(10), 가야곡왕주(9), 약주(8), 궁중술(7), 왕주(7), 역사(6), 전통(6), 청정지역(5), 깨끗한 맛(5), 숙취(4), 고급(4), 음식(3), 조청향(3), 단맛(2), 신맛(2), 목넘김(1), 쭈꾸미 볶음(1)
good
태그: 대표 막걸리, 인공 감미료 없음, 첨가물 없음, 가야양조장, 발효 기술, 숙성 기술, 진한 맛, 부드러움, 천연 과일향
중요도: 8, 9, 9, 7, 8, 8, 9, 8, 9
good
태그: 발효, 오랜 숙성, 저온 숙성, 기다림의 미학, 풍부한 맛, 풍부한 향, 농담금법, 정성

중요도: 8, 7, 7, 9, 9, 9, 6, 9
good


받아온 텍스트에서 가장 중요한건 처음에 태그 및 중요도가 따로 나온다는 점이고
line별로 받아서 파싱을 진행해야함


In [68]:
print(len(outputs))
print(len(failed))

148
13


In [74]:
failed

[4, 38, 39, 46, 57, 59, 66, 87, 88, 106, 122, 130, 138]

In [75]:
#실패한 dataframe만 가져옴
gpt_again=df1.loc[failed,:]
newindex=[]
failedindex=0
for row in gpt_again['텍스트 내용']:
  print(failed[failedindex])
  description =  row # 원하는 설명문을 입력하세요
  response = client.chat.completions.create(
  temperature=0.2,
  model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "주류 데이터를 검색하기 위한 태그를 설명문으로부터 추출하고 싶어. 술에 대한 정보를 포함하는 10자 이내의 태그와 각 태그의 중요도를 ,로 묶어서 출력하고 싶어. "},
    {"role": "system", "content": "또한 추출한 태그의 중요도를 1에서 10 사이의 수치로 표현해줘. 각 태그가 주관적인지 객관적인지 판단해서, 맛이나 향 등을 포함하거나 주관적이라면 높은 점수를 줘."},
    {"role": "user", "content": description},
  ]
  )
  print(response.choices[0].message.content)
  tempstr=response.choices[0].message.content
  if '태그: ' in tempstr and '중요도: ' in tempstr: #fine string
    # add tags
    lines=tempstr.split('\n')
    if "" in lines:
      lines.remove("")
    if len(lines)!=1 and '태그: ' in lines[0] and '중요도: ' in lines[1]:
      #good
      print("good")
      outputs[failed[failedindex]]=lines[0].replace("태그:","").replace(" ","")+"\n"+lines[1].replace("중요도:","").replace(" ","")
    else:
      newindex.append(failed[failedindex])
  elif '태그: ' in tempstr and len(tempstr.split('\n'))==1:
    print("oneline")
    p = re.compile('\(([^)]+)')
    tempstr=tempstr+"\n"
    for toint in p.findall(tempstr):
      tempstr=tempstr+toint+","
    tempstr=tempstr+"\n"
    tempstr=tempstr.replace(",\n","")
    tempstr=re.sub('\([^)]+\)','',tempstr)
    print(tempstr)
    outputs[failed[failedindex]]=tempstr.replace("태그:","").replace(" ","")
  else:
    newindex.append(failed[failedindex])
  failedindex=failedindex+1


4
태그: 삼양주, 탁주, 과일향, 새콤달콤함
중요도: 삼양주(10), 탁주(9), 과일향(8), 새콤달콤함(7)
good
38
태그: 깔끔한 탁주(10), 오양주(8), 발효(7), 숙성(7)
oneline
태그: 깔끔한 탁주, 오양주, 발효, 숙성
10,8,7,7
39
오곡, 풍부하게, 고소하게, 부드럽게, 발효, 생막걸리, 맛 변화, 효모, 당분, 보관 기간, 매력, 취향
46
태그: 천연 단맛(9), 깔끔한 산미(8), 목넘김(7), 막걸리(6)
oneline
태그: 천연 단맛, 깔끔한 산미, 목넘김, 막걸리
9,8,7,6
57
장밋빛의 로제 스파클링 와인, 케이크와 함께, 스위트 와인, 달콤한, 디저트
59
태그: 알콜45%, 증류주, 오크향
중요도: 10, 8, 7
good
66
태그: 바디감(8), 담백함(7), 과실향(9), 행복(10)
oneline
태그: 바디감, 담백함, 과실향, 행복
8,7,9,10
87
태그: 꿀맛(10), 달콤함(9), 스트레스 제로(7), 진정(6)
oneline
태그: 꿀맛, 달콤함, 스트레스 제로, 진정
10,9,7,6
88
1. 향미, 9
2. 단맛, 8
3. 도수, 7
4. 신맛, 6
5. 바디감, 7
106
태그: 탁주(10), 막걸리(9), 떠먹는 막걸리(8), 새콤한 맛(7), 단맛(7)
oneline
태그: 탁주, 막걸리, 떠먹는 막걸리, 새콤한 맛, 단맛
10,9,8,7,7
122
태그: 사인머스켓, N와인, 사인, 판매
중요도: 8, 7, 6, 5
good
130
#옥주프리미엄,8,#곡물향,7,#원기옥,9
138
태그: 안동국화주, 회곡안동소주, 국화꽃, 향긋한, 전통주, 증류식 소주, 맛, 은은한, 부드럽고 산뜻한, 목넘김, 참맛
중요도: 안동국화주(10), 회곡안동소주(8), 국화꽃(9), 향긋한(7), 전통주(9), 증류식 소주(6), 맛(8), 은은한(7), 부드럽고 산뜻한(8), 목넘김(7), 참맛(9)
good


In [76]:
newindex

[39, 57, 88, 130]

In [ ]:
newstr=re.sub('\([^)]+\)','',newstr)

In [77]:
#숫자만 존재하면 숫자만 리턴하고, 아니라면 문자열 파싱함

import re
finaltags=[]
finalweight=[]

for temp in outputs:
  line=temp.split("\n")
  tags=line[0].split(",")
  finaltags.append(tags)
  flag=True
  try:
    for s in line[1].split(","):
      if not s.isdigit():
        flag=False
        break
    if not flag:
      p = re.compile('\(([^)]+)')
      out=[]
      for toint in p.findall(line[1]):
        out.append(int(toint))
      print(out)
      finalweight.append(out)
    else:
      out=[]
      for toint in line[1].split(','):
        out.append(int(toint))
      print(out)
      finalweight.append(out)
  except:
    finalweight.append([])


[8, 9, 6, 7, 7, 8, 8, 5, 7, 6, 8, 6, 6, 6]
[8, 7, 7, 6, 6, 6, 5, 5, 4]
[10, 9, 5, 7, 8, 6, 9]
[9, 8, 9, 7, 7, 8, 7, 6, 8]
[10, 9, 8, 7]
[9, 9, 8, 8, 7, 7, 6, 6, 5, 5, 4]
[8, 9, 7, 7, 9, 8, 6]
[7, 6, 9, 5, 8, 8, 6, 7]
[8, 7, 6, 6, 7, 9, 10, 8, 7, 7, 7, 8, 7]
[9, 8, 7, 7, 9, 8, 7, 5, 5, 6]
[9, 8, 7, 7, 6, 6, 8, 8, 6, 7, 8]
[9, 7, 8]
[9, 8, 7, 6]
[8, 10, 7, 9]
[9, 8, 7, 7, 9, 10, 6, 6]
[7, 9, 6, 5, 4, 8, 8, 6, 7, 9, 8]
[9, 8, 7, 7, 6, 6, 8, 9, 6, 7, 8, 7, 8, 8, 7, 7]
[8, 7, 6, 9, 7, 8, 7, 9, 6, 8, 8]
[9, 8, 10, 7, 7, 6, 8, 6, 8, 9, 8, 5, 7, 9, 9, 8, 7, 6, 6]
[8, 7, 7, 6, 6, 5, 5, 9, 10, 8]
[8, 9, 7, 9]
[8, 9, 7, 10, 8, 6, 7]
[7, 8, 9, 6, 5, 7, 7, 6, 6, 8]
[7, 9, 8, 6, 5, 8, 7, 6, 6, 9, 7]
[10, 8, 7, 6, 9, 9]
[9, 8, 7, 6, 7, 6, 8, 7, 9, 7]
[8, 9, 7, 6, 7, 7, 9]
[7, 9, 8, 6, 9]
[8, 9]
[10, 9, 8, 9, 7, 8, 6, 9]
[7, 9, 8, 6, 5, 7, 8, 4, 7, 9, 8]
[7, 9, 8, 10, 6]
[7, 8, 9, 6, 8, 7, 9, 9]
[7, 7, 8, 6, 9, 8]
[8, 7, 6, 9]
[8, 6, 9, 7]
[7, 6, 8, 9]
[8, 6, 9, 7]
[10, 8, 7, 7]
[]
[9, 10, 7, 8, 8, 9,

In [79]:
print(len(finaltags))
print(len(finalweight))

148
148


In [ ]:
df1.head(10)

In [ ]:
response

ChatCompletion(id='chatcmpl-8M5GfzHrfNwe9hiBPoNrfTN56XsyN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='황매실주, 매실주, 매실주 원액, 황매실, 100% 매실주, 제주도 꿀, 전통 제조방법, 설탕 대신 꿀, 품평회, 수상', role='assistant', function_call=None, tool_calls=None))], created=1700274157, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=77, prompt_tokens=1285, total_tokens=1362))

In [81]:
df1['tags']=finaltags
df1['weight']=finalweight

In [82]:
df1.head()

,술 이름,텍스트 내용,tags,weight
0,고도리 복숭아 와인,GAP인증 복숭아로 만든 디저트 와인\n복숭아의 당도가 가장 높은 8월 말에 수확하...,"[와인, 복숭아, 디저트, 당도, 발효, 맛, 향, 냉동숙성, 아이스와인, 산, 아...","[8, 9, 6, 7, 7, 8, 8, 5, 7, 6, 8, 6, 6, 6]"
1,고도리 화이트,GAP인증 거봉 포도로 만든 신의 물방울\n포도의 당도가 가장 높은 10월 말에 수...,"[와인, 화이트와인, 과일향, 아로마틱, 달콤함, 산미, 고도리, 거봉포도, GAP인증]","[8, 7, 7, 6, 6, 6, 5, 5, 4]"
2,매실원주,황매실로 담근 100% 매실주\n\n국내 유일 기타 과실주를 첨가하지 않고 매실주 ...,"[매실주, 황매실, 100%, 제주도꿀, 전통제조방법, 향미, 품질인정]","[10, 9, 5, 7, 8, 6, 9]"
3,엠,화사한 메밀향과 스파이시한 목넘김\n반전매력 가득 메밀 증류주\n\n프레시하고 화사...,"[메밀향, 스파이시한, 반전매력, 프레시, 화사한, 정통, 시원한, 따뜻한, 특별한]","[9, 8, 9, 7, 7, 8, 7, 6, 8]"
4,술예쁘다,홍국쌀 특유의 과일향과 새콤달콤함이 인상적인 삼양주 탁주,"[삼양주, 탁주, 과일향, 새콤달콤함]","[10, 9, 8, 7]"


In [83]:
df1.to_csv('gpt_tag_weight.csv')
